In [ ]:
"""
Plot script.

Plots Tulip and Tulip/Noelle speedup v original openMP.
"""

%matplotlib inline

import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# List of benchmark directories
bmark_list = ['gesummv', 'fdtd-2d', 'heat-3d', 'atax', 'bicg', 'nussinov', 'lu', 'syrk', 'syr2k',
              'seidel-2d', 'gemm', '2mm', '3mm', 'doitgen', 'mvt', 'trmm', 'symm',
              'adi','jacobi-1d-imper', 'jacobi-2d-imper', 'gemver', 'covariance', 'correlation']

# Set this to True to print speedups
print_speedups = False

# Function to read the timing from a file
def read_time(file_path):
    try:
        with open(file_path, 'r') as file:
            return float(file.readline().strip())
    except:
        return None

def plot_bar_chart(compiler):
    # Initialize the data for the bar chart
    benchmarks = []
    tulip_speedups = []
    noelle_speedups = []

    # Iterate through each directory in the list
    for bmark in bmark_list:
        if os.path.exists(bmark):
            os.chdir(bmark)

            # Check if openmp.{compiler}.time exists
            if os.path.isfile(f'openmp.{compiler}.time'):
                openmp_time = read_time(f'openmp.{compiler}.time')

                # Check for tulip.{compiler}.time and tulip.{compiler}.noelle.time
                tulip_time = read_time(f'tulip.{compiler}.time')
                noelle_time = read_time(f'tulip.{compiler}.noelle.time')

                if tulip_time is not None or noelle_time is not None:
                    benchmarks.append(bmark)

                    tulip_speedup = openmp_time / tulip_time if tulip_time is not None else 0
                    noelle_speedup = openmp_time / noelle_time if noelle_time is not None else 0

                    tulip_speedups.append(tulip_speedup)
                    noelle_speedups.append(noelle_speedup)

                    if tulip_time is None:
                        print('Missing Tulip time for', bmark)
                    if noelle_time is None:
                        print('Missing Noelle time for', bmark)

                    if print_speedups:
                        print(f"Benchmark: {bmark}")
                        print(f"  Tulip Speedup: {tulip_speedup:.2f}")
                        print(f"  Noelle Speedup: {noelle_speedup:.2f}")

            os.chdir('..')

    # Plot the bar chart
    if benchmarks:
        width = 0.35  # the width of the bars
        fig, ax = plt.subplots(figsize=(10, 6))
        x = range(len(benchmarks))

        ax.bar(x, tulip_speedups, width, label=f'tulip.{compiler} speedup')
        ax.bar([p + width for p in x], noelle_speedups, width, label=f'tulip.{compiler}.noelle speedup')

        # Set the y-axis to use integer ticks
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.set_ylim(0, max(int(max(tulip_speedups + noelle_speedups, default=0)), 3) + 1)

        # Add gridlines at every integer
        ax.grid(True, which='both', axis='y', linestyle='-', color='gray', linewidth=0.5)
        ax.axhline(1, color='black', linewidth=1.5, label='Speedup = 1')

        ax.set_ylabel('Speedup relative to original OpenMP')
        ax.set_title(f'OpenMP on CPU ({compiler}): Speedup of OpenMP transpiled from CUDA v original OpenMP')
        ax.set_xticks([p + width / 2 for p in x])
        ax.set_xticklabels(benchmarks, rotation=70)
        ax.legend()

        # Save the plot to a file
        plt.tight_layout()
        plt.show()
        plt.close()

In [ ]:
compiler = 'clang'
plot_bar_chart(compiler)

In [ ]:
compiler = 'gcc'
plot_bar_chart(compiler)

In [ ]:
# Check results: Show any runs that had an output mismatch to sequential
try:
    with open('run.out', 'r') as file:
        for line in file:
            # Check if the line starts with "Output mismatch"
            if line.startswith('Output mismatch'):
                print(line.strip())
except FileNotFoundError:
    print("The file 'run.out' was not found.")